# DATA WRANGLING - KSI DATA

In [126]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [127]:
#import the data file for KSI - Killed or Seriously Injured from the Toronto Police Service webstie
data = pd.read_csv('KSI.csv')

## SETTING DATETIME INDEX

In [128]:
#As we notice the DATE columns is in UNIX timestamp, we will convert it into normal date format. 
#Since we are only analyzing accidents in Toronto, we only need one timezone.
data['DATE'] = data['DATE'].div(1000)
#convert to datetime from UNIX
data['DATE'] = pd.to_datetime(data['DATE'], unit = 's')
#remove Time from the DATE column and combine
data['DATE']= [d.date() for d in data['DATE']]
#drop the Hour, Index and Year columns
data = data.drop(columns=['Hour', 'Index_', 'YEAR', 'ObjectId'])
#convert the TIME column to proper format and combine with the DATE column
#convert the TIME column to be 4 digits using zfill
data['TIME'] = data['TIME'].apply(str)
data['TIME'] = [a.zfill(4) for a in data['TIME']]
#convert the TIME columns to represent time
data['TIME'] = [datetime.strptime(a, '%H%M') for a in data['TIME']]
data['TIME'] = [a.time() for a in data['TIME']]
#Set the data's Index to be datetime index
Index = pd.to_datetime(data['DATE'].apply(str)+' '+data['TIME'].apply(str))
data = data.set_index(Index)
#drop the original DATE and TIME columns
data= data.drop(columns = ['DATE', 'TIME'])
data = data.sort_index()

# Intial Data cleaning

In [129]:
#replace all empty values with NaN
data = data.replace(' ',np.nan )

In [130]:
#Dropping columns that are not of interest
data=data.drop(columns=['OFFSET','WardNum_X', 'WardNum_Y', 'Division', 'Division_X', 
                        'Division_Y','ACCLOC', 'LATITUDE', 'LONGITUDE', 'Hood_ID', 'FATAL_NO'])

### CREATING AN ADDRESS COLUMN FOR ACCIDENT INTERSECTION

In [131]:
#combine the STREET2 and STREET2 into a list
data['ADDRESS'] = data[['STREET1', 'STREET2']].values.tolist()
#Remove NaN values from ADDRESS (as there are NaN values in STREET2)
data['ADDRESS'] = data['ADDRESS'].apply(lambda x: [i for i in x if i == i])
#Sort the list to ensure correct count in case of reverse orders of STREET1 and STREET2
data['ADDRESS'] = data['ADDRESS'].apply(sorted)
#Make the Address column look nice
data['ADDRESS'] = data['ADDRESS'].apply(lambda x:  ' and '.join(x) if len(x)==2 else ''.join(x))

In [132]:
data=data.drop(columns=['STREET1', 'STREET2'])

### CLEANING AND FILLING DISTRICT/WARD/NEIGHBOURHOOD COLUMNS

In [133]:
#There are only 4 districts in Toronto, We will replace all Toronto East York with Toronto and East York
data['District'].unique()

array(['North York', 'Scarborough', 'Etobicoke York',
       'Toronto and East York', nan, 'Toronto East York'], dtype=object)

In [134]:
data['District'] = data['District'].replace('Toronto East York', 'Toronto and East York')

In [135]:
emptydistrict = data.loc[data['District'].isnull()]

In [136]:
emptydistrict['ADDRESS'].unique()

array(['EGLINTON AVE W and NORTHCLIFFE BLVD',
       'HIGHWAY 2A  E and LAWSON RD', 'BISHOP AVE and WILLOWDALE AVE',
       'F G GARDINER XY  E and LOWER JARVIS ST', '176 LAWRENCE AVE E',
       'FINCH  AVE W and MARTIN GROVE RD', 'BRIMLEY RD and PROGRESS AVE'],
      dtype=object)

In [137]:
mapdict = {'EGLINTON AVE W and NORTHCLIFFE BLVD': 'Toronto and East York',
       'HIGHWAY 2A  E and LAWSON RD':'Scarborough',
       'BISHOP AVE and WILLOWDALE AVE':'North York',
       'F G GARDINER XY  E and LOWER JARVIS ST': 'Toronto and East York',
       '176 LAWRENCE AVE E': 'North York',
       'FINCH  AVE W and MARTIN GROVE RD':'Etobicoke York',
       'BRIMLEY RD and PROGRESS AVE': 'Scarborough'}

In [138]:
#map from the mapdict above to clear up the NaN values in District
data['District'] = data['District'].fillna(data['ADDRESS'].map(mapdict))

In [139]:
#make sure that there is no NaN values in District columns
data.District.unique()

array(['North York', 'Scarborough', 'Etobicoke York',
       'Toronto and East York'], dtype=object)

In [140]:
#There should only be 25 Ward, we need to fix those wards that were numbered higher than 25
W1 = data.loc[(data['WardNum']>25) | (data['WardNum']==0)]

In [141]:
W1.ADDRESS.unique()

array(['EGLINTON AVE E and OSWEGO RD',
       'FINCH AVE E and VICTORIA PARK AVE', 'KANE AVE and ROGERS RD',
       'EGLINTON AVE W and ENNERDALE RD',
       'EGLINTON AVE E and PHARMACY AVE', 'DIXON RD and MARTIN GROVE RD',
       'BATHURST ST and CARPENTER RD', 'DIXON RD and ISLINGTON AVE',
       '160 MARKHAM RD', 'EGLINTON AVE W and PEVERIL HILL  N',
       'COLLEGE ST and OSSINGTON AVE', 'CARLTON ST and YONGE ST',
       'BATHURST Stre and DREWRY Aven', 'GRANDRAVINE DR and SENTINEL RD',
       'DUFFERIN ST and ROGERS RD', 'EGLINTON AVE W and ELMSTHORPE AVE',
       'BELLAMY RD N and EGLINTON AVE E', 'BAYVIEW AVE and FINCH AVE E',
       'BAY ST and DUNDAS ST W', 'KINGSTON RD and MORNINGSIDE AVE',
       'PEARD RD and VICTORIA PARK AVE', 'KEELE St and ST REGIS Cres',
       'BARBADOS BLVD and EGLINTON AVE E', 'DIXON Rd and ROYAL YORK Rd',
       'COXWELL AVE and DANFORTH AVE', 'BATHURST ST and DUNDAS ST W',
       'BEVERLY HILLS Driv and JANE Stre',
       'LOWER JARVIS ST and THE 

In [142]:
mapdict2 = {'EGLINTON AVE E and OSWEGO RD':20, 'FINCH AVE E and VICTORIA PARK AVE':17,'KANE AVE and ROGERS RD':9, 
            'EGLINTON AVE W and ENNERDALE RD':9,'EGLINTON AVE E and PHARMACY AVE':21, 'DIXON RD and MARTIN GROVE RD':1,
            'BATHURST ST and CARPENTER RD':6, 'DIXON RD and ISLINGTON AVE':1,'160 MARKHAM RD':11, 
            'EGLINTON AVE W and PEVERIL HILL  N':12,'COLLEGE ST and OSSINGTON AVE':9, 'CARLTON ST and YONGE ST':11,
            'BATHURST Stre and DREWRY Aven':6, 'GRANDRAVINE DR and SENTINEL RD':6,'DUFFERIN ST and ROGERS RD':9, 
            'EGLINTON AVE W and ELMSTHORPE AVE':8,'BELLAMY RD N and EGLINTON AVE E':21, 'BAYVIEW AVE and FINCH AVE E':17,
            'BAY ST and DUNDAS ST W':11,'KINGSTON RD and MORNINGSIDE AVE':25, 'PEARD RD and VICTORIA PARK AVE':19,
            'KEELE St and ST REGIS Cres':6, 'BARBADOS BLVD and EGLINTON AVE E':21,'DIXON Rd and ROYAL YORK Rd':2, 
            'COXWELL AVE and DANFORTH AVE':14,'BATHURST ST and DUNDAS ST W':10, 'BEVERLY HILLS Driv and JANE Stre':7,
            'LOWER JARVIS ST and THE ESPLANADE':13, 'BALLIOL ST and MOUNT PLEASANT RD':12,'AVENUE RD and EGLINTON AVE W':8, 
            'SCARLETT RD and ST CLAIR AVE W':5,'3807 VICTORIA PARK Ave':17, 'DUNDAS Stre W and WEST MALL Cres':3,
            'ALAMEDA Aven and EGLINTON Aven W':12, 'SLOANE Aven and VICTORIA PARK Aven':16,'HURONIA GT and MCCOWAN RD':21, 
            'CASSANDRA BLVD and VICTORIA PARK AVE':16,'1420 VICTORIA PARK AVE':16, 'ATLAS AVE and EGLINTON AVE W':8,
            'FINCH AVE W and KEELE ST':7, 'RANLEIGH AVE and YONGE ST':8,'DIXON RD and KIPLING AVE':2, 'EGLINTON Ave E and MIDLAND Ave':20,
            'BURNHAMTHORPE RD and KIPLING AVE':2,'401 C E MCCOWAN RD RAMP and MCCOWAN RD':21, 
            'JANE ST and SHEPPARD AVE W':7,'COSBURN AVE and COXWELL AVE':14, 'KEELE ST and STEELES AVE W':14,
            'LAVINGTON DR and MARTIN GROVE RD':1, 'PAGET RD and VICTORIA PARK AVE':21,'ARGYLE ST and DOVERCOURT RD':10, 
            'EGLINTON AVE E and LEBOVIC AVE':20,'MCLEVIN Ave and NEILSON Rd':23, 'HAVENDALE RD and MIDLAND AVE':22,
            'MCCOWAN RD and TOWN CENTRE CRT':24, 'KEELE ST and TORO RD W':6,'CHAPLIN CRES and EGLINTON AVE W':12, 
            'BAY ST and QUEEN ST W':10,'DANFORTH RD and EGLINTON AVE E':21, 'CONSUMERS RD and VICTORIA PARK AVE':17,
            '2160 EGLINTON AVE W and CALEDONIA Road':8, 'MIDLAND AVE and PASSMORE AVE':22,'GERRARD St E and YONGE St':13, 
            'MCNICOLL Aven and MIDLAND Aven':23,'MCNICOLL AVE and MIDLAND AVE':23, 'DON MILLS RD and OVERLEA BLVD':16,
            'MUSGRAVE ST and VICTORIA PARK AVE':20,'BROADWAY Aven and MOUNT PLEASANT Road':15, 
            'BAY ST and KING ST W':13,'WILLIAM CARSON CRES and YONGE ST':15, 'NORTHCLIFFE BLVD and ROGERS RD':12,
            'DUNDAS ST W and MC CAUL ST':11, 'BAY ST and WELLINGTON ST W':10,'FARM GREENWAY and VICTORIA PARK AVE':16,
            'EGLINTON AVE E and VICTORIA PARK AVE':16, 'CASCI AVE and COXWELL AVE':19,
            'MIDLAND AVE and SOUTH SHIELDS AVE':22, 'ROGERS Road and SILVERTHORN Aven':9,
            'COXWELL Aven and QUEEN Stre E':14, 'LAWRENCE Aven E and MCCOWAN Road':21,
            'MANOR RD E and MOUNT PLEASANT RD':12, 'COXWELL Ave and ROBBINS Ave':19,'BAYVIEW Aven and CUMMER Aven':18,
            'EGLINTON AVE W and OLD PARK RD':8,'EGLINTON AVE W and MARTIN GROVE RD':2, 'DUNDAS ST W and PALMERSTON AVE':11,
            'BAYVIEW AVE and BAYVIEW MEWS LANE':18, 'DUFFERIN ST and EGLINTON AVE W':12,'BATHURST ST and EGLINTON AVE W':12, 
            'DUNDAS ST W and SPADINA AVE':11,'LAWRENCE AVE E and VICTORIA PARK AVE':21, 'CHERRY ST and LAKE SHORE BLVD E':10,
            'BAYVIEW S ROSEDALE VL RAMP and ROSEDALE VALLEY RD':13,'BIGGIN CRT and VICTORIA PARK AVE':21, 
            '401 C W BAYVIEW RAMP and BAYVIEW AVE':17,'LAWRENCE Aven E and MORNINGSIDE Aven':25, 
            'MILL ST and YONGE ST':15,'MCNICOLL AVE and VICTORIA PARK AVE':22, 'JANE ST and WILLIAM CRAGG DR':6,
            'ARNCLIFFE CRES and VICTORIA PARK AVE':21, 'BAY ST and ST JOSEPH ST':11,'KEELE ST and MURRAY ROSS PKWY':6, 
            'DOVERCOURT RD and DUNDAS ST W':10,'EGLINTON Aven W and MARTIN GROVE Road':2, 'COXWELL AVE and O CONNOR DR':19,
            'JANE ST and WILSON AVE':7, 'BLOOR ST E and SHERBOURNE ST N':13,'EGLINTON AVE W and MARLEE AVE':8, 
            'EGLINTON AVE W and OAKWOOD AVE':8,'LESLIE Stre and LESMILL Road':15, 'EGLINTON Aven E and MIDLAND Aven':20,
            'BAY ST and WELLESLEY ST W':11, 'BATHURST ST and STEELES AVE W':11,'3655 KEELE ST':6, 
            'MCLEVIN AVE and NEILSON RD':23,'BATHURST ST and FINCH AVE W':18, 'DUNDAS Stre W and ELIZABETH Stre':11,
            'BLOOR Stre W and SHAVER Aven N':3, 'HUNTINGWOOD Driv and MIDLAND Aven':22,'FINCH Aven E and MIDLAND Aven':23, 
            'DUNDAS St W and UNIVERSITY Ave':11,'MOUNT PLEASANT RD and TED ROGERS  WAY':13, 'DIXON RD and ST PHILLIPS RD':2,
            'EGLINTON AVE E and MOUNT PLEASANT RD':12, 'DUFFERIN ST and PRESTON RD':9,'ARLETA AVE and SHEPPARD AVE W':7,
            'DVP S EASTERN AV W RAMP S and EASTERN DIVERSION AVE E':14,'BLOOR ST W and LAUREL AVE':2, 
            'DUFFERIN Stre and EGLINTON Aven W':12,'DUNDAS St W and YONGE St':13, 'MOUNT PLEASANT RD and PENROSE RD':15,
            'BAYVIEW AVE and POTTERY RD':11, 'EGLINTON AVE W and LOCKSLEY AVE':12,'BAY ST and COLLEGE ST':11, 
            'BATHURST ST and WILD GINGERWAY Aven':6,'BIMBROK RD and EGLINTON AVE E':21, 
            'DUFFERIN St and EVERSFIELD Rd':9,'ELLESMERE Road and MCCOWAN Road':21, 'OAKWOOD AVE and ROGERS RD':12,
            'DIXON RD and MAPLEBRANCH PATH':1, 'DUNDAS ST E and YONGE ST':13,'YONGE ST and YORK MILLS RD':8, 
            'CHURCH ST and THE ESPLANADE':19,'FARMSTEAD Rd and LESLIE St':15, 'EGLINTON Ave E and GILDER Dr':21,
            'FINCH AVE E and MIDLAND AVE':23, '519 DUNDAS ST W':10, '2600 LESLIE St':15,
            'BLOOR ST E and ROSEDALE VALLEY RD':13, 'DUFFERIN ST and KEYWEST AVE':12,'1300 YONGE St':12, 
            'DAVISVILLE AVE and MOUNT PLEASANT RD':15,'ROWENA DR and VICTORIA PARK AVE':16, 
            'BIRCHMOUNT RD and EGLINTON AVE E':21,'LAWRENCE AVE E and MCCOWAN RD':21, 'KINGSTON RD and MARKHAM RD':20,
            'DUNDAS ST W and YONGE ST':13, 'JANE Stre and SHEPPARD Aven W':7,'MCCOWAN RD and MELDAZY DR':24, 
            'DUNDAS ST W and MANNING AVE':10,'PACHINO BLVD and VICTORIA PARK AVE':16, 'BATHURST ST and BRIDGMAN AVE':12,
            'MIDLAND AVE and STEELES AVE E':23, 'ELLESMERE RD and MCCOWAN RD':21,'SLOANE AVE and VICTORIA PARK AVE':16, 
            'EGLINTON AVE E and IONVIEW RD':21,'MCCOWAN RD and ST ANDREWS RD':24,'BAY ST and TEMPERANCE ST W':13, 
            'DUNDAS ST W and NEILSON DR':13,'255 MORNINGSIDE AVE':25, 'BELLAMY Road N and NELSON Stre':24,
            'BRAEMAR Aven and EGLINTON Aven W':12, 'DUNDAS ST W and HURON ST':10,'EGLINTON AVE W and HEDDINGTON AVE':8, 
            'HIGHWAY 401 C  E and MARKHAM RD':24,'WILSON AVE and YONGE ST':8, 'MCGLASHAN RD and YONGE ST':15,
            'EGLINTON Aven W and MARLEE Aven':8, 'SHEPPARD AVE E and VICTORIA PARK AVE':22,
            'BERKELEY Stre and THE ESPLANADE':13, 'SHAFTESBURY AVE and YONGE ST':11,'BATHURST ST and DEWLANE DR':18, 
            'DUPLEX Aven and EGLINTON Aven W':8,'CARLYLE ST and DUNDAS ST W':11, 'BAYVIEW Ave and MCKEE Ave':18,
            'DEWSON ST and OSSINGTON AVE':11, 'DIXON RD and GOLFWOOD HTS Heig':2,'BAYLAWN DR and MIDLAND AVE':22, 
            'BELLAMY Road N and BURNVIEW Cres':24,'ANTIBES DR and BATHURST ST':6, 'DIXON Road and MARTIN GROVE Road':1,
            'BAY ST and GRENVILLE ST':13, 'GRANDRAVINE DR and KEELE ST':6,'ELM ST and YONGE ST':13, 
            'ROGERS RD and SILVERTHORN AVE':9,'HOLLAND PARK AVE and OAKWOOD AVE':9, 'JANE Stre and WILSON Aven':7,
            'DUNDAS ST W and GRACE ST':11, 'BATHURST Stre and DUNDAS Stre W':10, 'EGLINTON AVE E and FALMOUTH AVE':21, 
            'ADELAIDE St W and BAY St':10,'NEILSON Rd and TAPSCOTT Rd':25,'PARKWOODS VILLAGE Driv and VICTORIA PARK Aven':16,
            'FINCH AVE E and NEILSON RD':25, 'ROEHAMPTON AVE and YONGE ST':12,'THE ESPLANADE Stre E and YONGE ST':10, 
            'COXWELL AVE and QUEEN ST E':14,'EGLINTON AVE E and MIDLAND AVE':20, 'EGLINTON Aven E and YONGE Stre':8,
            'BUCKLAND Road and SHEPPARD Aven W':6, 'ELLESMERE Rd and MCCOWAN Rd':6,'BELLAMY Road N and EGLINTON Aven E':21, 
            'BAYVIEW AVE and ROSEDALE VALLEY RD':13,'BLOOR Stre W and LAUREL Aven':2, 'LAKE SHORE BLVD W and PALACE PIER CRT':3,
            'DUNDAS Stre W and SHAW Stre':10, 'CALEDONIA RD and EGLINTON AVE W':9,'COSBURN Ave and COXWELL Ave':14, 
            'BATHURST ST and MASCOT PL':18,'NEILSON RD and SHEPPARD AVE E':23, '1437 VICTORIA PARK AVE':20,
            'LAWRENCE AVE E and LESLIE ST':16, 'DUNDAS ST W and GORE VALE AVE':11,'SUNRISE AVE and VICTORIA PARK AVE':20, 
            'DAVENPORT RD and OSSINGTON AVE':12,'3300 MIDLAND AVE':22, 'BRIMLEY RD and EGLINTON AVE E':21,
            'DUFFERIN ST and QUEEN ST W':4, 'BAYVIEW AVE and NESBITT DR':15,'CLYDESDALE DR and VICTORIA PARK AVE':17,
            'SOUTHMEAD RD and VICTORIA PARK AVE':20, 'COLLEGE ST and YONGE ST':11,'3133 BAYVIEW AVE and BYNG AVE':17, 
            'TERRAVIEW BLVD and VICTORIA PARK AVE':16,'DUFFERIN St and SPRINGHURST Ave':4, 'BAY St and FRONT St W':13,
            'BAYVIEW AVE and FELDBAR CRT':17, 'EGLINTON AVE E and KENNEDY RD':20,'ALAMEDA Ave and EGLINTON Ave W':12, 
            'AVENUE RD and MACPHERSON AVE':11,'DUNDAS ST W and OSSINGTON AVE':9, 'EGLINTON AVE W and HENNING AVE':8,
            'OLD WESTON RD and ROGERS RD':8, 'CELESTINE DR and DIXON RD':2,'ORCHARD VIEW BLVD and YONGE ST':12, 
            'BISCAYNE BLVD and VICTORIA PARK Ave':21,'GLENHOLME AVE and ROGERS RD':9, 'WILSON Ave and YONGE St':8,
            'GERRARD ST E and YONGE ST':13, 'RIVERVIEW HTS and SCARLETT RD':1,'KILCULLEN CASTLE GT and MIDLAND AVE':22, 
            'EGLINTON AVE E and SINNOTT RD':20,'DUNDAS ST W and PRINCE EDWARD DR N':2, 'DUFFERIN St and GIBSON St':9,
            '1420 VICTORIA PARK Ave':16, 'DON ROADWAY and LAKE SHORE BLVD E':14,'BLOOR St W and THE EAST MALL':11, 
            'BEATH St and MORNINGSIDE Ave':24,'BRAEMAR AVE and EGLINTON AVE W':12, 'LESLIE ST and LESMILL RD':15,
            'GLENGROVE AVE E and YONGE ST':15, 'EGLINTON Ave W and OAKWOOD Ave':8,'DUPONT St and OSSINGTON Ave':9, 
            'VICTORIA PARK AVE and WOODALE AVE':20,'EGLINTON AVE W and PARK HILL RD':12, 'MORECAMBE GT and VICTORIA PARK AVE':17,
            'DUFFERIN St and HOLMESDALE Rd':9, 'DUFFERIN ST and JOE SHUSTER  WAY':4,'EGLINTON AVE E and MASON RD':20, 
            'LOWER JARVIS St and THE ESPLANADE':13,'EGLINTON Ave W and WINONA Dr':8, 'OLD YORK MILLS RD and YONGE ST':15,
            'LOWER SHERBOURNE Stre and THE ESPLANADE':10,'EDGE PARK Ave and VICTORIA PARK Ave':20, '1092 KEELE ST':6,
            '401 C E MORNINGSIDE AVE S RAMP and MORNINGSIDE AVE':25,'DUNDAS ST W and ROYAL YORK RD':3, 
            'CHILTERN HILL RD and EGLINTON AVE W':12,'955 BAY ST':13, 'MOUNT PLEASANT RD and TED ROGERS WAY':13,
            'FRONT ST and YONGE ST':13, 'CLAREMONT ST and DUNDAS ST W':13,'O CONNOR DR and VICTORIA PARK AVE':20,
            'EGLINTON AVE E and MCCOWAN RD':21,'EGLINTON AVE W and NORTHCLIFFE BLVD':12, 'BRIMLEY RD and HIGHWAY 401 C  W':23,
            'PARMA CRT and VICTORIA PARK AVE':20,'ST BEDES RD and VICTORIA PARK AVE':20, '3035 YONGE ST':8,
            'BENLEIGH  DR and MCCOWAN RD':21,'GARDENTREE ST and MORNINGSIDE AVE':24, 'DUNDAS ST W and KENSINGTON AVE':11,
            'AUGUSTA AVE and DUNDAS ST W':10, '1999 LESLIE ST':16,'BERNER TRL and NEILSON RD':23,'COXWELL AVE and GERRARD ST E':19,
            '1140 LESLIE ST':16,'GLENCREST BLVD and VICTORIA PARK AVE':19, 'ST CLAIR AVE W and WINONA DR':12,
            'FINCH AVENUE WEST and ISLINGTON AVENUE':1, 'BAYVIEW AVE and CUMMER AVE':18,'EGLINTON AVE E and MARKHAM RD':24,
            'EGLINTON AVE E and EGLINTON SQ':20, 'BROADWAY AVE and REDPATH AVE':12,'EGLINGTON AVE and HEDDINGTON AVE':8,
            'MERTON ST and MT PLEASANT RD':15,'DUNDAS ST W and STERLING RD':9, '6030 BATHURST ST':6,
            'BLOOR ST W and OSSINGTON AVE':9,'MERTON ST and MOUNT PLEASANT RD':15, 'BAYVIEW AVE and SHEPPARD AVE E':18,
            'ALBERTUS AVE and YONGE ST':8, 'MIDLAND AVE and SILVER STAR BLVD':22,'FINCH AVE W and TANGIERS RD':6, 
            '3157 BAYVIEW AVE':17,'COMBERMERE DR and VICTORIA PARK AVE':21,'CROW TRL and NEILSON RD':25, 
            'ST CLAIR AVE E and VICTORIA PARK AVE':19,'FRONT ST E and YONGE ST':13, 'BAYVIEW AVE and BOWAN CRT':17,
            'LAWRENCE AVE E and YONGE ST':8, 'COXWELL AVE and ROBBINS AVE':19,'2058 DUFFERIN ST':9, 
            'VICTORIA PARK AVE and YORK MILLS RD':21,'BAYVIEW AVE and WYCLIFFE CRES':17, '1500 EGLINTON AVE W':8, 
            'DUFFERIN ST and HUNTER AVE':9,'BELLAMY RD N and LAWRENCE AVE E':21,'1468 VICTORIA PARK AVE':16,
            'MARTIN GROVE RD and RICHGROVE DR':1,'DUNDAS ST W and ROXTON RD':10, 'COXWELL AVE and EARL HAIG AVE':19,
            'EGLINTON AVE W and MENIN RD':8,'HUNTINGWOOD DR and VICTORIA PARK AVE':22,'BAYVIEW AVE and CITATION DR':18, 
            'SHELDRAKE BLVD and YONGE ST':8,'EGLINTON AVE E and HAKIMI  AVE':20,'OSSINGTON AVE and PENDRITH ST':11,
            'EGLINTON AVE E and YONGE ST':8, 'PATRICK BLVD and VICTORIA PARK AVE':22,'BEACHELL ST and EGLINTON AVE E':24,
            '2632 EGLINTON AVE E':21, 'JANE ST and TROUTBROOKE DR':7,'BAY ST and HAGERMAN ST':13, '58 THE ESPLANADE':13,
            'NORTHOVER ST and SHEPPARD AVE W':7,'DON VALLEY PARKWAY  N and F G GARDINER XY  E':14,
            'HIGHWAY 401 C  W and VICTORIA PARK AVE':21,'ELLESMERE RD and MORNINGSIDE AVE':24,
            'ACORES AVE and OSSINGTON AVE':9,'MIDLAND AVE and SHEPPARD AVE E':22, 'EGLINTON AVE E and WINTER AVE':20,
            'DUFFERIN ST and KING ST W':4, 'EGLINTON AVE E and PRUDHAM GT':20,'LOWER SHERBOURNE ST and THE ESPLANADE':10,
            'ALBERTUS AVE and YONGE  ST':8,'FRONT ST W and YONGE ST':13, 'BAY ST and GROSVENOR ST':13,
            'MIDLAND  AVE and SHEPPARD  AVE E':22, 'ADELAIDE ST W and BAY ST':10,'MAGELLAN DR and SHEPPARD AVE W':6,
            '401 C E VICTORIA PR N RAMP and VICTORIA PARK AVE':21,'EGLINTON AVE W and RUSSELL HILL RD':12,
            'MARTIN GROVE RD and TOLLINGTON RD':2,'COMMONWEALTH AVE and EGLINTON AVE E':21,'ERSKINE AVE and YONGE ST':15, 
            'BEVERLEY ST and DUNDAS ST W':10,'GOULD ST and YONGE  ST':13, 'DUPLEX AVE and EGLINTON AVE W':8,
            'BROADWAY AVE and MOUNT PLEASANT RD':15,'BAYVIEW AVE and BURLEIGH HEIGHTS DR':17,
            '2 m South of JANE ST TROUTBROOKE DR':7, 'DUPONT ST and OSSINGTON AVE':9,'EGLINTON AVE W and MIRANDA AVE':8,
            'DUNDAS ST W and ELIZABETH ST':11,'COXWELL AV              (542) and DANFORTH AV':14,
            'LYNVALLEY CRES and VICTORIA PARK AVE':16,'BATHURST ST and BRENTHALL AVE':18,'EDGE PARK AVE and VICTORIA PARK AVE':20,
            'CLINTWOOD GT and VICTORIA PARK AVE':21,'EGLINTON AVE E and TORRANCE RD':20,'HARBORD ST and OSSINGTON  AVE':11, 
            'YONGE BLVD and YONGE ST':8,'CHILTERN HILL RD and EGLINTON  AVE W':12,'LAWRENCE AVE W and YONGE ST':8,
            'COXWELL AVE and MORTIMER AVE':14, '401 C W NEILSON RAMP RD and NEILSON RD':23,'100 SUNRISE AVE':16,
            'EGLINTON AVE E and ROSEMOUNT DR W of':21, 'RANLEIGH  AVE and YONGE  ST':8,'483 DUNDAS ST W':10, 
            'HALLAM ST and OSSINGTON  AVE':11,'3230 BAYVIEW AVE':18,'ROSELAWN AVE and YONGE ST':8, 
            'BATHURST ST and ELLERSLIE AVE':18,'BATHURST STREET and STEELES AVENUE WEST':18,'300 COXWELL AVE':14,
            'DUNDAS STREET WEST and HURON STREET':10,'BATHURST ST and CEDARCROFT BLVD':6,'BEVERLY HILLS DR and JANE ST':7, 
            'BAY  ST and GROSVENOR ST':13,'JONESVILLE CRES and VICTORIA PARK AVE':21, 'COXWELL AVE and EASTERN AVE':14,
            'LAWRENCE AVE E and MORNINGSIDE AVE':25,'BLOOR ST W and KIPLING  AVE':3,'CASTLE KNOCK RD and EGLINTON AVE W':8,
            'BAYVIEW AVE and BAYVIEW S 401 C W RAMP':17,'FARMCREST  DR and VICTORIA PARK AVE':22,
            'GEARY AVE and OSSINGTON  AVE':9, 'DUFFERIN ST and GIBSON ST':9,'EGLINTON AVE E and LESLIE ST':16, 
            'AUGUSTA  AVE and DUNDAS ST W':10,'KEELE STREET and STEELES AVENUE WEST':7, 'JUNCTION RD and OLD WESTON RD':5,
            'DIXON RD and KIPLING AVE':2, 'CHESTNUT ST and DUNDAS ST W':10,'EGLINTON AVE E and ROSEMOUNT DR':21, 
            'ESQUIRE RD and VICTORIA PARK AVE':17,'COXWELL AVE GERRARD ST E':19,'CRESCENT TOWN RD and VICTORIA PARK AVE':19, 
            'COLLEGE ST and OSSINGTON AV':9,'CARLTON ST and YONGE  ST':11,'NORTHUMBERLAND ST and OSSINGTON AVE':9, 
            'FOUR WINDS  DR and KEELE ST':6,'BAYVIEW AVE and STEELES AVE E':17, 'HIGHWAY 2A  E and LAWSON RD':19,
            'BISHOP AVE and WILLOWDALE AVE':18,'F G GARDINER XY  E and LOWER JARVIS ST':10}

In [143]:
#Fix up WardNum column
data['WardNum'] = np.where((data['WardNum']>25) | (data['WardNum']==0), np.nan, data['WardNum'])
data['WardNum'] = data['WardNum'].fillna(data['ADDRESS'].map(mapdict2))
data['WardNum']=data['WardNum'].astype(int)

In [144]:
data.loc[data['Neighbourh'].isnull()].ADDRESS.unique()

array(['MARKHAM Road and STEELES',
       'PICKERING TOWN LINE and STEELES AVE E',
       'STEELES AVE W and TANGREEN CRT',
       'HIGHWAY 27  N and STEELES Ave W', 'NINTH LINE and STEELES AVE E',
       'OLD KENNEDY RD and STEELES AVE E',
       'CARPENTER RD and STEELES AVE W', '4472 STEELES AVE E',
       'SILVER STAR BLVD and STEELES AVE E',
       'STEELES AVE W and STEINWAY BLVD',
       'STEELES Aven E and WARDEN Aven',
       'ISLINGTON AVE and STEELES AVE W', 'KLONDIKE Dr and STEELES Ave W',
       'STEELES AVE E and TOWNSEND RD', 'STEELES AVE E and TAPSCOTT RD',
       'HIGHWAY 27 S and STEELES AVE W', '2800 STEELES AVE W',
       'SIGNET DR and STEELES AVE W', 'STEELES AVE E and WARDEN AVE',
       'FINCH AVE E and PICKERING TOWN LINE'], dtype=object)

In [145]:
mapdict3 = {'MARKHAM Road and STEELES':'Milliken (130)',
       'PICKERING TOWN LINE and STEELES AVE E':'Clairlea-Birchmount (120)',
       'STEELES AVE W and TANGREEN CRT':'Newtonbrook West (36)',
       'HIGHWAY 27  N and STEELES Ave W':'West Humber-Clairville (1)', 'NINTH LINE and STEELES AVE E':'Rouge (131)',
       'OLD KENNEDY RD and STEELES AVE E':'Milliken (130)',
       'CARPENTER RD and STEELES AVE W':'Westminster-Branson (35)', '4472 STEELES AVE E':'Milliken (130)',
       'SILVER STAR BLVD and STEELES AVE E':'Milliken (130)',
       'STEELES AVE W and STEINWAY BLVD':'West Humber-Clairville (1)',
       'STEELES Aven E and WARDEN Aven':'Steeles (116)',
       'ISLINGTON AVE and STEELES AVE W':'Humber Summit (21)', 'KLONDIKE Dr and STEELES Ave W':'Humber Summit (21)',
       'STEELES AVE E and TOWNSEND RD':'Hillcrest Village (48)', 'STEELES AVE E and TAPSCOTT RD':'Rouge (131)',
       'HIGHWAY 27 S and STEELES AVE W':'West Humber-Clairville (1)', '2800 STEELES AVE W':'York University Heights (27)',
       'SIGNET DR and STEELES AVE W':'Humber Summit (21)', 'STEELES AVE E and WARDEN AVE':'Steeles (116)',
       'FINCH AVE E and PICKERING TOWN LINE':'Townline'}

In [146]:
data['Neighbourh'] = data['Neighbourh'].fillna(data['ADDRESS'].map(mapdict3))

### REPLACING ALL NAN VALUES WITH 'N' (for YES/NO columns) AND 'Unknown/Other' (for categorical columns)

In [147]:
#Replace Some NaN value with No
data[['PEDESTRIAN','CYCLIST', 'AUTOMOBILE','MOTORCYCLE', 'TRUCK', 'PASSENGER',
      'SPEEDING', 'AG_DRIV', 'REDLIGHT', 'ALCOHOL', 'DISABILITY','TRSN_CITY_',
      'EMERG_VEH']] = data[['PEDESTRIAN','CYCLIST', 'AUTOMOBILE','MOTORCYCLE', 'TRUCK', 'PASSENGER',
      'SPEEDING', 'AG_DRIV', 'REDLIGHT', 'ALCOHOL', 'DISABILITY','TRSN_CITY_','EMERG_VEH']].replace(np.nan, 'No')
data[['LOCCOORD', 'ROAD_CLASS', 'TRAFFCTL', 'VISIBILITY', 'LIGHT', 'RDSFCOND', 'INVTYPE', 'INVAGE', 
      'INJURY', 'INITDIR', 'VEHTYPE', 'MANOEUVER', 'DRIVACT', 'DRIVCOND','PEDTYPE', 'PEDCOND', 'CYCLISTYPE',
      'CYCACT','PEDACT','CYCCOND']] = data[['LOCCOORD', 'ROAD_CLASS', 'TRAFFCTL', 'VISIBILITY', 'LIGHT', 'RDSFCOND', 'INVTYPE', 'INVAGE', 
      'INJURY', 'INITDIR', 'VEHTYPE', 'MANOEUVER', 'DRIVACT', 'DRIVCOND','PEDTYPE', 'PEDCOND', 'CYCLISTYPE',
      'CYCACT','PEDACT','CYCCOND']].replace(np.nan, 'Unknown/Other')
data[['LOCCOORD', 'ROAD_CLASS', 'TRAFFCTL', 'VISIBILITY', 'LIGHT', 'RDSFCOND', 'INVTYPE', 'INVAGE', 
      'INJURY', 'INITDIR', 'VEHTYPE', 'MANOEUVER', 'DRIVACT', 'DRIVCOND','PEDTYPE', 'PEDCOND', 'CYCLISTYPE',
      'CYCACT','PEDACT','CYCCOND']] = data[['LOCCOORD', 'ROAD_CLASS', 'TRAFFCTL', 'VISIBILITY', 'LIGHT', 'RDSFCOND', 'INVTYPE', 'INVAGE', 
      'INJURY', 'INITDIR', 'VEHTYPE', 'MANOEUVER', 'DRIVACT', 'DRIVCOND','PEDTYPE', 'PEDCOND', 'CYCLISTYPE',
      'CYCACT','PEDACT','CYCCOND']].replace(['Other', 'Unknown', 'unknown'], 'Unknown/Other')

## FINAL DATAFRAME AFTER INITIAL CLEANING

In [185]:
#Dropping Vehicle Owner since it is not of interest
data = data[data['INVTYPE']!='Vehicle Owner']
data

,ACCNUM,ROAD_CLASS,District,WardNum,LOCCOORD,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,...,TRSN_CITY_,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,Neighbourh,ADDRESS
2008-01-03 16:23:00,1015097,Major Arterial,North York,15,Intersection,Traffic Signal,Snow,Daylight,Loose Snow,Non-Fatal Injury,...,No,No,No,No,No,No,No,No,Mount Pleasant East (99),BAYVIEW AVE and EGLINTON AVE E
2008-01-03 16:23:00,1015097,Major Arterial,North York,15,Intersection,Traffic Signal,Snow,Daylight,Loose Snow,Non-Fatal Injury,...,No,No,No,No,No,No,No,No,Mount Pleasant East (99),BAYVIEW AVE and EGLINTON AVE E
2008-01-04 15:40:00,1015121,Major Arterial,North York,16,Mid-Block,No Control,Clear,Daylight,Dry,Non-Fatal Injury,...,No,No,No,No,No,No,No,No,Banbury-Don Mills (42),UPJOHN RD and YORK MILLS RD
2008-01-04 15:40:00,1015121,Major Arterial,North York,16,Mid-Block,No Control,Clear,Daylight,Dry,Non-Fatal Injury,...,No,No,No,No,No,No,No,No,Banbury-Don Mills (42),UPJOHN RD and YORK MILLS RD
2008-01-05 18:26:00,1012986,Major Arterial,Scarborough,20,Intersection,No Control,Rain,Dark,Wet,Fatal,...,No,No,No,No,No,No,No,No,Eglinton East (138),EGLINTON AVE E and OSWEGO RD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-26 18:53:00,8002363781,Major Arterial,Etobicoke York,3,Mid-Block,No Control,Clear,"Dark, artificial",Dry,Fatal,...,No,No,No,No,No,No,No,No,Islington-City Centre West (14),4968 DUNDAS ST W
2018-12-26 18:53:00,8002363781,Major Arterial,Etobicoke York,3,Mid-Block,No Control,Clear,"Dark, artificial",Dry,Fatal,...,No,No,No,No,No,No,No,No,Islington-City Centre West (14),4968 DUNDAS ST W
2018-12-31 06:50:00,8002388715,Major Arterial,Scarborough,21,Intersection,Traffic Signal,Clear,"Dark, artificial",Wet,Non-Fatal Injury,...,No,No,Yes,Yes,Yes,Yes,No,No,Bendale (127),BRIMLEY RD and PROGRESS AVE
2018-12-31 06:50:00,8002388715,Major Arterial,Scarborough,21,Intersection,Traffic Signal,Clear,"Dark, artificial",Wet,Non-Fatal Injury,...,No,No,Yes,Yes,Yes,Yes,No,No,Bendale (127),BRIMLEY RD and PROGRESS AVE


## WRANGLING WITH THE FINAL DATA

### LOOKING AT ACCIDENT LOCATION (ACCNUM=ACCIDENT CODE)

In [175]:
#Count the number of accidents happening in different kinds of road
data[['ACCNUM','ROAD_CLASS']].drop_duplicates().groupby(['ROAD_CLASS']).count()

,ACCNUM
ROAD_CLASS,
Collector,280
Expressway,236
Laneway,3
Local,219
Major Arterial,2922
Minor Arterial,727
Pending,3
Unknown/Other,15


In [176]:
#Count the number of accident in different District
data[['ACCNUM','District']].drop_duplicates().groupby(['District']).count()

,ACCNUM
District,
Etobicoke York,943
North York,787
Scarborough,1016
Toronto and East York,1659


In [177]:
#Count the number of accident in different Neighbourhoods
data[['ACCNUM','Neighbourh']].drop_duplicates().groupby(['Neighbourh']).count().sort_values('ACCNUM', ascending=False)

,ACCNUM
Neighbourh,
Waterfront Communities-The Island (77),168
West Humber-Clairville (1),135
Bay Street Corridor (76),100
Woburn (137),89
South Riverdale (70),81
...,...
Danforth (66),5
Elms-Old Rexdale (5),4
Maple Leaf (29),4


### Looking at Addresses with High Number of Accidents over the year (>4) 

In [178]:
#Count the number of different District
addressgroup = data[['ACCNUM','ADDRESS']].drop_duplicates().groupby(['ADDRESS']).count()
#grouping addresses with high number of incident, more than 4
k = list(addressgroup.loc[addressgroup.ACCNUM>4].index)

In [179]:
highaccident = data.loc[data['ADDRESS'].isin(k)]

In [180]:
#There are 234 streets with high number of incident
len(highaccident.ACCNUM.unique())

234

In [181]:
#Most likely impacts seen in highaccident roads
highaccident[['ACCNUM','IMPACTYPE']].drop_duplicates().groupby(['IMPACTYPE']).count()

,ACCNUM
IMPACTYPE,
Angle,19
Approaching,5
Cyclist Collisions,17
Other,4
Pedestrian Collisions,112
Rear End,20
SMV Other,18
Sideswipe,3
Turning Movement,36


In [182]:
#Observe the traffic situation at the scene of the locations with high frequency incident
highaccident[['ACCNUM','INVTYPE']].drop_duplicates().groupby(['INVTYPE']).count()

,ACCNUM
INVTYPE,
Cyclist,17
Driver,224
Driver - Not Hit,2
Motorcycle Driver,20
Motorcycle Passenger,3
Other Property Owner,9
Passenger,65
Pedestrian,111
Truck Driver,13


### Observing Driver and Pedestrian Action 

In [186]:
data['DRIVACT'].value_counts()

Unknown/Other                   5245
Driving Properly                2836
Failed to Yield Right of Way    1081
Lost control                     613
Improper Turn                    380
Disobeyed Traffic Control        300
Following too Close              186
Exceeding Speed Limit            158
Speed too Fast For Condition     132
Improper Lane Change              92
Improper Passing                  69
Wrong Way on One Way Road          6
Speed too Slow                     2
Name: DRIVACT, dtype: int64

In [187]:
data['PEDACT'].value_counts()

Unknown/Other                            9200
Crossing with right of way                663
Crossing, no Traffic Control              478
Crossing without right of way             262
On Sidewalk or Shoulder                   167
Running onto Roadway                      157
Crossing, Pedestrian Crossover             40
Person Getting on/off Vehicle              31
Coming From Behind Parked Vehicle          25
Playing or Working on Highway              21
Walking on Roadway with Traffic            20
Walking on Roadway Against Traffic         13
Crossing marked crosswalk without ROW      11
Pushing/Working on Vehicle                 10
Person Getting on/off School Bus            2
Name: PEDACT, dtype: int64